### Problem Statement

text here

image here

### 1) Imports

In [ ]:
from __future__ import print_function           # Use print() instead of print
from __future__ import division                 # 1 / 2 = 0.5, 1 // 2 = 0
import numpy as np                              # for numpy
import torch                                    # import PyTorch module 
import torch.nn as nn                           # neural network module
from torch.utils.data import DataLoader         # dataloader abstracts sampling of minibatch
import torchvision.datasets as dset             # standard machine learning datasets
import torchvision.transforms as tform          # transforms for data augmentation
from torch.autograd import Variable             # auto differentiation
import sd_utils as utils                        # some python helper functions ( see: sd_utils.py ) 

### 2) Hyper Parmaters and Initializations

### 3) Download and Preprocess Dataset

### 4) Create DataLoader objects

### 5) Define Network

### 6) Define Loss function and Optimizer

### 7) Training Loop

### 8) Testing Loop

### 9) Save / Load Model